# Facebook爬蟲
這裡簡單介紹一下爬蟲的邏輯再開始爬進入正題吧！
- 爬取文章內容
 - 貼文人ID
 - 貼文人暱稱
 - 貼文時間
 - 貼文類型
 - 貼文內容
 - 心情：分為Like, Haha, Sad, Wow, Angry...etc.
 - 留言數
 - 分享數
 - 貼文連結


- 爬取留言內容
 - 留言人ID
 - 留言人暱稱
 - 留言時間
 - 回覆對象ID
 - 回覆對象暱稱
 - 貼文連結
 

# 載入使用套件

In [1]:
import pandas as pd
import re, time, requests, datetime, gc
from selenium import webdriver
from bs4 import BeautifulSoup

# 經常用到的函數
Facebook請用戶「登入/註冊」的彈窗會反覆出現，在這裡先寫好嘗試點擊的函數提供使用

In [ ]:
# 點擊不要現在註冊帳號
def ClickNotNow():
    try:
        driver.find_element_by_xpath('//a[@id="expanding_cta_close_button"]').click()
    except:
        time.sleep(0.1)

# 截取塗鴉墻貼文資訊並比對先前資料
如果每次都把所有貼文抓下來，無疑會浪費程式效能，在這裡我將載入先前的資料，並將各篇貼文分成以下三類：

- **新貼文**：截取貼文內容與留言
- **舊貼文但有新留言**：截取所有留言但只將新留言加入資料庫
- **舊貼文也無新留言**：不處理

可想而知，如果設定排程每天執行，大多數資料都會是「舊貼文也無新留言」的類型，加入這個判斷機制將大幅節省系統效能

In [2]:
# 截取塗鴉墻上貼文的留言數
def Wall_PostComments(i):
    try:
        Comments = i.find('a', {'data-testid':'UFI2CommentsCount/root'}).text.split(' ',2)[0]
        if 'K' in Comments:
            Comments = int(float(Comments.split('K')[0])*1000)
        else:
            Comments = int(Comments)
    except:
        Comments = 0
    return Comments

# 截取塗鴉墻上貼文的連結
def Wall_PostLink(i):
    Link = 'https://www.facebook.com' + i.find('a',{'class':'_5pcq'}).attrs['href'].split('?',2)[0]
    return Link

# 截取塗鴉墻上貼文的發佈時間
def Wall_PostTime(i):
    try:
        Time = i.find('abbr').attrs['title']
        Time = datetime.datetime.strptime(Time, '%m/%d/%y, %I:%M %p')
        Time = Time.strftime("%Y-%m-%d %H:%M")
    except:
        Time = 'Not Post'
    return Time

def CarwlList(urls, n, Posts):
    CheckList = pd.DataFrame(columns = ['Link','Time','Comments']) 
    for i in urls:
        driver.get(i)        
        for i in range(n):
            try:
                time.sleep(1)
                driver.find_element_by_css_selector('a.pam.uiBoxLightblue.uiMorePagerPrimary').click() # 加載更多貼文的按鈕
            except:
                time.sleep(1)
            # 這裡會跳出要我們登入的大畫面，找到「稍後再說」的按鈕並點擊
            ClickNotNow()
            soup = BeautifulSoup(driver.page_source)
            for i in soup.find_all('div', {'class':'_5pcr userContentWrapper'}):
                CheckList = pd.concat([CheckList, pd.DataFrame(data = [{'Link':Wall_PostLink(i),
                                                                        'Time':Wall_PostTime(i),
                                                                        'Comments':Wall_PostComments(i)}],
                                                               columns = ['Link','Time','Comments'])],
                                       ignore_index = True)
    
    CarwlList = pd.merge(left = CheckList,
                         right = Posts.loc[:,['Link','Comments']],
                         how='left',
                         on='Link',
                         suffixes=('_c', '_p'),
                         indicator=True)
    CarwlList = pd.concat([CarwlList.loc[CarwlList._merge == 'left_only', :], # 新貼文
                           CarwlList.loc[CarwlList.Comments_c > CarwlList.Comments_p, :]], # 舊貼文但有新留言
                          ignore_index=True)
    CarwlList = CarwlList.drop_duplicates(subset = 'Link',
                                          keep= 'first',
                                          inplace = False)
    return CarwlList

# 爬取特定貼文的內容與留言
## 展開貼文與留言

In [9]:
# 關閉彈窗
def ClosePopup():
    time.sleep(1)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(1)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    ClickNotNow()
    driver.find_element_by_xpath('//div[@class="_5pcr userContentWrapper"]//a[@data-testid="UFI2CommentsCount/root"]').click()
    time.sleep(1)
    ClickNotNow()
    time.sleep(0.5)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(0.5)
    ClickNotNow()
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(0.5)
    ClickNotNow()

# 選擇按照New或Oldest來顯示留言(選擇最相關或所有留言都不會會真的顯示所有留言)
def ClickOldest():
    time.sleep(2)
    driver.find_element_by_xpath('//a[@data-testid="UFI2ViewOptionsSelector/link"]').click()
    time.sleep(1)
    try:
        driver.find_element_by_partial_link_text('Comments shown in chronological order with the oldest comments at the top.').click()
    except:
        try:
            driver.find_element_by_partial_link_text('New comments and those with new replies appear at the top.').click()
        except:
            print('Plz, Check this post arragne type!')
    time.sleep(1)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    ClickNotNow()
    time.sleep(1)

# 偵測是否有「更多留言」（第一層），若有則點擊
def MoreComments():
    k = len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]'))
    # 卡住的次數超過100次就停止
    l = 0
    while (k != 0) & (l <= 300):
        print('Click More Comments： ' + str(l) + ' times.')
        try:
            driver.find_element_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]').click()
            time.sleep(1)
            # 若彈出註冊視窗點擊not now!
            ClickNotNow()
            k = len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]'))
        except:
            time.sleep(0.1)
        finally:
            l += 1
    time.sleep(1)

# 偵測是否有「更多回覆」（第二層），若有則點擊
def MoreReplies():
    k = len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]'))
    # 卡住的次數超過100次就停止
    l = 0
    while (k != 0) & (l <= 300):
        l += 1
        print('Click More Replies： ' + str(l) + ' times.')
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        driver.find_element_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]').click()
        time.sleep(0.5)
        k = len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]'))
    time.sleep(1)
    
# 偵測是否有「展開留言」（第二層留言內容），若有則點擊
def SeeMore():
    k = len(driver.find_elements_by_xpath('//a[@class="_5v47 fss"]'))
    # 卡住的次數超過100次就停止
    l = 0
    while (k != 0) & (l <= 1000):
        l += 1
        print('Click See More ： ' + str(l) + ' times.')
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        driver.find_element_by_xpath('//a[@class="_5v47 fss"]').click()
        time.sleep(0.5)
        k = len(driver.find_elements_by_xpath('//a[@class="_5v47 fss"]'))
    time.sleep(1)
    
# 組合以上函數，將貼文內容展開
def PostExpand():
    ClosePopup()
    ClickOldest()
    MoreComments()
    MoreReplies()
    SeeMore()

## 爬取貼文內容與留言資訊

In [4]:
# 貼文留言數
def Post_Name(userContent):
    return userContent.find('img').attrs['aria-label']

# 貼文留言數
def Post_ID(userContent):
    return userContent.find('a', {'class':'_5pb8 o_c3pynyi2g _8o _8s lfloat _ohe'}).attrs['href'].split('/?ref')[0].split('https://www.facebook.com/')[1]

# 貼文時間
def Post_Time(userContent):
    try:
        Time = userContent.find('abbr').attrs['title']
        Time = datetime.datetime.strptime(Time, '%m/%d/%y, %I:%M %p')
        Time = Time.strftime("%Y-%m-%d %H:%M")
    except:
        Time = "Error, Please check this post's condiction!" 
    return Time

# 貼文內容
def Post_Content(userContent):
    try:
        Content = userContent.find('div', {'class':'_5pbx userContent _3576'}).text
    except:
        Content = "There's No Text Content!"
    return Content

# 貼文留言數
def Post_Comments(userContent):
    try:
        CommentCounts = userContent.find('a', {'data-testid':'UFI2CommentsCount/root'}).text.split(' ',2)[0]
        if 'K' in CommentCounts:
            CommentCounts = int(float(CommentCounts.split('K')[0])*1000)
        else:
            CommentCounts = int(CommentCounts)
    except:
        CommentCounts = 0 
    return CommentCounts

# 貼文分享數
def Post_Shares(userContent):
    try:
        ShareCounts = userContent.find('span', {'class':'_355t _4vn2'}).text.split(' ',2)[0]
        if 'K' in ShareCounts:
            ShareCounts = int(float(ShareCounts.split('K')[0])*1000)
        else:
            ShareCounts = int(ShareCounts)
    except:
        ShareCounts = 0
    return ShareCounts

# 貼文按讚心情數
def Post_Likes(userContent):
    try:
        LikeCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_LIKE'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in LikeCounts:
            LikeCounts = int(float(LikeCounts.split('K')[0])*1000)
        else:
            LikeCounts = int(LikeCounts)
    except:
        LikeCounts = 0
    return LikeCounts

# 貼文愛心心情數
def Post_Loves(userContent):
    try:
        LoveCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_LOVE'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in LoveCounts:
            LoveCounts = int(float(LoveCounts.split('K')[0])*1000)
        else:
            LoveCounts = int(LoveCounts)
    except:
        LoveCounts = 0
    return LoveCounts

# 貼文哈哈心情數
def Post_Hahas(userContent):
    try:
        HahaCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_HAHA'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in HahaCounts:
            HahaCounts = int(float(HahaCounts.split('K')[0])*1000)
        else:
            HahaCounts = int(HahaCounts)
    except:
        HahaCounts = 0
    return HahaCounts

# 貼文Wow心情數
def Post_Wows(userContent):
    try:
        WowCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_WOW'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in WowCounts:
            WowCounts = int(float(WowCounts.split('K')[0])*1000)
        else:
            WowCounts = int(WowCounts)
    except:
        WowCounts = 0
    return WowCounts

# 貼文Sad心情數
def Post_Sads(userContent):
    try:
        SadCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_SORRY'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in SadCounts:
            SadCounts = int(float(SadCounts.split('K')[0])*1000)
        else:
            SadCounts = int(SadCounts)
    except:
        SadCounts = 0
    return SadCounts

# 貼文Angry心情數
def Post_Angrys(userContent):
    try:
        AngryCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_ANGER'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in AngryCounts:
            AngryCounts = int(float(AngryCounts.split('K')[0])*1000)
        else:
            AngryCounts = int(AngryCounts)
    except:
        AngryCounts = 0
    return AngryCounts

# 文章內容與互動摘要
def PostInfo(soup):
    # 貼文區
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    PostContent = pd.DataFrame(data = [{'Name':Post_Name(userContent),
                                        'ID':Post_ID(userContent),
                                        'Time':Post_Time(userContent),
                                        'Content':Post_Content(userContent),
                                        'Comments':Post_Comments(userContent),
                                        'Shares':Post_Shares(userContent),
                                        'Likes':Post_Likes(userContent),
                                        'Loves':Post_Loves(userContent),
                                        'Hahas':Post_Hahas(userContent),
                                        'Wows':Post_Wows(userContent),
                                        'Sads':Post_Sads(userContent),
                                        'Angrys':Post_Angrys(userContent),
                                        'Updatetime':datetime.datetime.now().strftime("%Y-%m-%d %H:%M"),
                                        'Link':driver.current_url}],
                            columns = ['Name', 'ID', 'Time', 'Content', 'Comments', 'Shares', 'Likes', 'Loves', 'Hahas', 'Wows', 'Sads', 'Angrys', 'Updatetime', 'Link'])
    return PostContent

# 留言內容
def Comment_Content(element):
    try:
        Content = element.find('span', {'dir':'ltr'}).text
    except:
        Content = 'img'
    return Content

def CommentsInfo(soup):  
    PostComments = pd.DataFrame()
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    try:
        for i in userContent.select('ul._7a9a > li'):
            # 先抓留言並放在Comment
            Comment = pd.DataFrame(data=[{'ID':i.find('a', {'class':' _3mf5 _3mg0'}).attrs['data-hovercard'].split('id=',2)[1],
                                          'Name':i.find('img').attrs['alt'],
                                          'Time':datetime.datetime.strptime(i.find('abbr',{'class':'livetimestamp'}).attrs['data-tooltip-content'], '%A, %B %d, %Y at %I:%M %p').strftime("%Y-%m-%d %H:%M"),
                                          'Content':Comment_Content(i),
                                          'RepID':userContent.find('div', {'class':'_5pcp _5lel _2jyu _232_'}).attrs['id'].split(';')[0].split('feed_subtitle_')[-1],
                                          'RepName':userContent.find('img').attrs['aria-label'],
                                          'Link':driver.current_url}],
                                   columns = ['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName', 'Link'])
            PostComments = pd.concat([PostComments, Comment], ignore_index=True)
            # 留言的留言
            for j in i.findAll('div', {'data-testid':'UFI2Comment/root_depth_1'}):
                Comment = pd.DataFrame(data=[{'ID':j.find('a', {'class':' _3mf5 _3mg1'}).attrs['data-hovercard'].split('id=',2)[1],
                                              'Name':j.find('img').attrs['alt'],
                                              'Time':datetime.datetime.strptime(j.find('abbr',{'class':'livetimestamp'}).attrs['data-tooltip-content'], '%A, %B %d, %Y at %I:%M %p').strftime("%Y-%m-%d %H:%M"),
                                              'Content':Comment_Content(j),
                                              'RepID':i.find('a', {'class':' _3mf5 _3mg0'}).attrs['data-hovercard'].split('id=',2)[1],
                                              'RepName':i.find('img').attrs['alt'],
                                              'Link':driver.current_url}],
                                       columns = ['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName', 'Link'])
                PostComments = pd.concat([PostComments, Comment], ignore_index=True)
        PostComments['Updatetime'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
    except:
        print('Crawl Comments Failed!')
    return PostComments

# 更新貼文與留言資訊
將最新抓到的貼文併回Post表格，並以ID與Time作為關鍵(key)值，取最新抓取的資料(Updatetime)

In [5]:
def UpdateData(DateFrame_o,DateFrame_n):
    DataFrame = pd.concat([DateFrame_o, DateFrame_n], ignore_index=True)
    DataFrame = DataFrame.sort_values(by = 'Updatetime', ascending = False)
    DataFrame = DataFrame.drop_duplicates(subset = ['ID', 'Time', 'Content'],
                                          keep= 'first',
                                          inplace = False)
    return DataFrame

# 爬資料
## 讀取先前資料
若在桌面偵測不到檔案，就創造新的dataframe

In [6]:
try:
    Posts = pd.read_pickle('C:/Users/TL_Yu/Desktop/Posts.plk')
except:
    Posts = pd.DataFrame(columns=['Name', 'ID', 'Time', 'Content', 'Comments', 'Shares', 'Likes', 'Loves', 'Hahas', 'Wows', 'Sads', 'Angrys', 'Updatetime', 'Link'])
try:
    Comments = pd.read_pickle('C:/Users/TL_Yu/Desktop/Comments.plk')
except:
    Comments = pd.DataFrame(columns = ['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName', 'Link', 'Updatetime'])

print(Posts.shape)
print(Comments.shape)
urls = ['https://www.facebook.com/tstartel/',
        'https://www.facebook.com/chtmobile/',
        'https://www.facebook.com/taiwanmobile/',
        'https://www.facebook.com/fareastone/',
        'https://www.facebook.com/Aptg.tw/']

(253, 14)
(17447, 8)


## 產出待爬清單

In [7]:
driver = webdriver.Chrome()
driver.get('https://www.facebook.com/')
time.sleep(1)
driver.find_element_by_partial_link_text('English').click()

In [8]:
CarwlList = CarwlList(urls=urls, n=10, Posts = Posts)
CarwlList

C:\Users\TL_Yu\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,Link,Time,CommentCounts_c,CommentCounts_p,_merge
0,https://www.facebook.com/tstartel/photos/a.854...,2019-05-28 04:24,5,NaN,left_only
1,https://www.facebook.com/tstartel/photos/a.854...,2019-05-27 21:18,40,NaN,left_only
6,https://www.facebook.com/tstartel/videos/32195...,2019-05-14 21:00,143,NaN,left_only
10,https://www.facebook.com/tstartel/photos/a.251...,2019-05-06 00:43,164,NaN,left_only
23,https://www.facebook.com/tstartel/photos/a.413...,2019-04-21 21:00,26,NaN,left_only
24,https://www.facebook.com/tstartel/photos/a.239...,2019-04-19 20:51,67,NaN,left_only
37,https://www.facebook.com/tstartel/posts/318084...,2019-04-07 21:11,109,NaN,left_only
45,https://www.facebook.com/tstartel/videos/58966...,2019-03-31 21:03,0,NaN,left_only
46,https://www.facebook.com/chtmobile/posts/27210...,2019-05-23 02:00,16,NaN,left_only
47,https://www.facebook.com/chtmobile/photos/a.26...,2019-05-22 02:00,8,NaN,left_only


如同上面的說明，當CheckLists「merge」欄位為「left_only」時表示該貼文是新貼文；

而當「merge」欄位為「both」，但「Comments_c」的值大於「Comments_p」時，表示該貼文是舊資料但有新留言。

以下將截取這兩類的貼文作為爬取清單

## 執行爬蟲

In [ ]:
for i in CarwlList.Link:
    print('Dealing with: ' + i)
    driver.get(i)
    try:
        PostExpand()
        print('Expand Succed!')
        try:
            soup = BeautifulSoup(driver.page_source)
            time.sleep(1)
            nPost = PostInfo(soup)
            nComments = CommentsInfo(soup)
            
            Posts = UpdateData(DateFrame_o = Posts, DateFrame_n = nPost)
            print('Update PostInfo complete!')
            nComments = CommentsInfo(soup)
            Comments = UpdateData(DateFrame_o = Comments,DateFrame_n = nComments)
            print('Update CommentsInfo complete!')
        except:
            print('Update Post or CommentsInfo Failed!')
    except:
        print('This post can not expand')
    gc.collect()
    print('Time Log: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + '\n\n------------------')

Dealing with: https://www.facebook.com/tstartel/photos/a.854369967922385/3306199622739395/
Click More Comments： 0 times.
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-05-28 20:48:58

------------------
Dealing with: https://www.facebook.com/tstartel/photos/a.854369967922385/3305536929472331/
Click More Comments： 0 times.
Click More Replies： 1 times.
Click More Replies： 2 times.
Click More Replies： 3 times.
Click More Replies： 4 times.
Click More Replies： 5 times.
Click More Replies： 6 times.
Click See More ： 1 times.
Click See More ： 2 times.
Click See More ： 3 times.
Click See More ： 4 times.
Expand Succed!
Update PostInfo complete!
Update CommentsInfo complete!
Time Log: 2019-05-28 20:49:24

------------------
Dealing with: https://www.facebook.com/tstartel/videos/321955001812559/
Click More Comments： 0 times.
Click More Comments： 1 times.
Click More Comments： 2 times.
Click More Comments： 3 times.
Click More Comments： 4 times.
Click More Comme

Click More Comments： 234 times.
Click More Comments： 235 times.
Click More Comments： 236 times.
Click More Comments： 237 times.
Click More Comments： 238 times.
Click More Comments： 239 times.
Click More Comments： 240 times.
Click More Comments： 241 times.
Click More Comments： 242 times.
Click More Comments： 243 times.
Click More Comments： 244 times.
Click More Comments： 245 times.
Click More Comments： 246 times.
Click More Comments： 247 times.
Click More Comments： 248 times.
Click More Comments： 249 times.
Click More Comments： 250 times.
Click More Comments： 251 times.
Click More Comments： 252 times.
Click More Comments： 253 times.
Click More Comments： 254 times.
Click More Comments： 255 times.
Click More Comments： 256 times.
Click More Comments： 257 times.
Click More Comments： 258 times.
Click More Comments： 259 times.
Click More Comments： 260 times.
Click More Comments： 261 times.
Click More Comments： 262 times.
Click More Comments： 263 times.
Click More Comments： 264 times.
Click Mo

Click More Comments： 62 times.
Click More Comments： 63 times.
Click More Comments： 64 times.
Click More Comments： 65 times.
Click More Comments： 66 times.
Click More Comments： 67 times.
Click More Comments： 68 times.
Click More Comments： 69 times.
Click More Comments： 70 times.
Click More Comments： 71 times.
Click More Comments： 72 times.
Click More Comments： 73 times.
Click More Comments： 74 times.
Click More Comments： 75 times.
Click More Comments： 76 times.
Click More Comments： 77 times.
Click More Comments： 78 times.
Click More Comments： 79 times.
Click More Comments： 80 times.
Click More Comments： 81 times.
Click More Comments： 82 times.
Click More Comments： 83 times.
Click More Comments： 84 times.
Click More Comments： 85 times.
Click More Comments： 86 times.
Click More Comments： 87 times.
Click More Comments： 88 times.
Click More Comments： 89 times.
Click More Comments： 90 times.
Click More Comments： 91 times.
Click More Comments： 92 times.
Click More Comments： 93 times.
Click Mo

This post can not expand
Time Log: 2019-05-28 21:03:08

------------------
Dealing with: https://www.facebook.com/chtmobile/photos/a.264830403546609/2625898320773127/
This post can not expand
Time Log: 2019-05-28 21:03:12

------------------
Dealing with: https://www.facebook.com/notes/%E4%B8%AD%E8%8F%AF%E9%9B%BB%E4%BF%A1%E8%A1%8C%E5%8B%95%E5%AD%B8%E5%9C%92/%E4%B8%AD%E8%8F%AF%E9%9B%BB%E4%BF%A1%E6%A6%AE%E7%8D%B2%E8%A1%8C%E5%8B%95%E7%B6%B2%E9%80%9F%E4%B8%89%E5%86%A0%E7%8E%8B%E5%8F%B0%E4%B8%AD%E6%B0%B8%E7%A6%8F%E5%8F%8A%E9%AB%98%E9%9B%84%E4%BA%94%E7%A6%8F%E9%96%80%E5%B8%82%E9%99%90%E5%AE%9A%E5%8A%A0%E7%A2%BC%E6%8A%BD%E7%8D%8E%E6%B4%BB%E5%8B%95-%E4%B8%AD%E7%8D%8E%E5%90%8D%E5%96%AE/2706220829404971/
This post can not expand
Time Log: 2019-05-28 21:03:17

------------------
Dealing with: https://www.facebook.com/chtmobile/posts/2621480854548207
This post can not expand
Time Log: 2019-05-28 21:03:20

------------------
Dealing with: https://www.facebook.com/chtmobile/photos/a.264830403546609/

Click More Comments： 136 times.
Click More Comments： 137 times.
Click More Comments： 138 times.
Click More Comments： 139 times.
Click More Comments： 140 times.
Click More Comments： 141 times.
Click More Comments： 142 times.
Click More Comments： 143 times.
Click More Comments： 144 times.
Click More Comments： 145 times.
Click More Comments： 146 times.
Click More Comments： 147 times.
Click More Comments： 148 times.
Click More Comments： 149 times.
Click More Comments： 150 times.
Click More Comments： 151 times.
Click More Comments： 152 times.
Click More Comments： 153 times.
Click More Comments： 154 times.
Click More Comments： 155 times.
Click More Comments： 156 times.
Click More Comments： 157 times.
Click More Comments： 158 times.
Click More Comments： 159 times.
Click More Comments： 160 times.
Click More Comments： 161 times.
Click More Comments： 162 times.
Click More Comments： 163 times.
Click More Comments： 164 times.
Click More Comments： 165 times.
Click More Comments： 166 times.
Click Mo

...

Click More Comments： 271 times.
Click More Comments： 272 times.
Click More Comments： 273 times.
Click More Comments： 274 times.
Click More Comments： 275 times.
Click More Comments： 276 times.
Click More Comments： 277 times.
Click More Comments： 278 times.
Click More Comments： 279 times.
Click More Comments： 280 times.
Click More Comments： 281 times.
Click More Comments： 282 times.
Click More Comments： 283 times.
Click More Comments： 284 times.
Click More Comments： 285 times.
Click More Comments： 286 times.
Click More Comments： 287 times.
Click More Comments： 288 times.
Click More Comments： 289 times.
Click More Comments： 290 times.
Click More Comments： 291 times.
Click More Comments： 292 times.
Click More Comments： 293 times.
Click More Comments： 294 times.
Click More Comments： 295 times.
Click More Comments： 296 times.
Click More Comments： 297 times.
Click More Comments： 298 times.
Click More Comments： 299 times.
Click More Comments： 300 times.
Click More Replies： 1 times.
Expand Succ

Click More Comments： 195 times.
Click More Comments： 196 times.
Click More Comments： 197 times.
Click More Comments： 198 times.
Click More Comments： 199 times.
Click More Comments： 200 times.
Click More Comments： 201 times.
Click More Comments： 202 times.
Click More Comments： 203 times.
Click More Comments： 204 times.
Click More Comments： 205 times.
Click More Comments： 206 times.
Click More Comments： 207 times.
Click More Comments： 208 times.
Click More Comments： 209 times.
Click More Comments： 210 times.
Click More Comments： 211 times.
Click More Comments： 212 times.
Click More Comments： 213 times.
Click More Comments： 214 times.
Click More Comments： 215 times.
Click More Comments： 216 times.
Click More Comments： 217 times.
Click More Comments： 218 times.
Click More Comments： 219 times.
Click More Comments： 220 times.
Click More Comments： 221 times.
Click More Comments： 222 times.
Click More Comments： 223 times.
Click More Comments： 224 times.
Click More Comments： 225 times.
Click Mo

Click More Comments： 147 times.
Click More Comments： 148 times.
Click More Comments： 149 times.
Click More Comments： 150 times.
Click More Comments： 151 times.
Click More Comments： 152 times.
Click More Comments： 153 times.
Click More Comments： 154 times.
Click More Comments： 155 times.
Click More Comments： 156 times.
Click More Comments： 157 times.
Click More Comments： 158 times.
Click More Comments： 159 times.
Click More Comments： 160 times.
Click More Comments： 161 times.
Click More Comments： 162 times.
Click More Comments： 163 times.
Click More Comments： 164 times.
Click More Comments： 165 times.
Click More Comments： 166 times.
Click More Comments： 167 times.
Click More Comments： 168 times.
Click More Comments： 169 times.
Click More Comments： 170 times.
Click More Comments： 171 times.
Click More Comments： 172 times.
Click More Comments： 173 times.
Click More Comments： 174 times.
Click More Comments： 175 times.
Click More Comments： 176 times.
Click More Comments： 177 times.
Click Mo

Click More Comments： 80 times.
Click More Comments： 81 times.
Click More Comments： 82 times.
Click More Comments： 83 times.
Click More Comments： 84 times.
Click More Comments： 85 times.
Click More Comments： 86 times.
Click More Comments： 87 times.
Click More Comments： 88 times.
Click More Comments： 89 times.
Click More Comments： 90 times.
Click More Comments： 91 times.
Click More Comments： 92 times.
Click More Comments： 93 times.
Click More Comments： 94 times.
Click More Comments： 95 times.
Click More Comments： 96 times.
Click More Comments： 97 times.
Click More Comments： 98 times.
Click More Comments： 99 times.
Click More Comments： 100 times.
Click More Comments： 101 times.
Click More Comments： 102 times.
Click More Comments： 103 times.
Click More Comments： 104 times.
Click More Comments： 105 times.
Click More Comments： 106 times.
Click More Comments： 107 times.
Click More Comments： 108 times.
Click More Comments： 109 times.
Click More Comments： 110 times.
Click More Comments： 111 tim

Click More Comments： 19 times.
Click More Comments： 20 times.
Click More Comments： 21 times.
Click More Comments： 22 times.
Click More Comments： 23 times.
Click More Comments： 24 times.
Click More Comments： 25 times.
Click More Comments： 26 times.
Click More Comments： 27 times.
Click More Comments： 28 times.
Click More Comments： 29 times.
Click More Comments： 30 times.
Click More Comments： 31 times.
Click More Comments： 32 times.
Click More Comments： 33 times.
Click More Comments： 34 times.
Click More Comments： 35 times.
Click More Comments： 36 times.
Click More Comments： 37 times.
Click More Comments： 38 times.
Click More Comments： 39 times.
Click More Comments： 40 times.
Click More Comments： 41 times.
Click More Comments： 42 times.
Click More Comments： 43 times.
Click More Comments： 44 times.
Click More Comments： 45 times.
Click More Comments： 46 times.
Click More Comments： 47 times.
Click More Comments： 48 times.
Click More Comments： 49 times.
Click More Comments： 50 times.
Click Mo

In [10]:
Posts = Posts.reset_index(drop=True)
Posts = Posts.loc[:,['Name', 'ID', 'Time', 'Content', 'Comments', 'Shares', 'Likes', 'Loves', 'Hahas', 'Wows', 'Sads', 'Angrys', 'Updatetime', 'Link']]
Posts

,Name,ID,Time,Content,Comments,Shares,Likes,Loves,Hahas,Wows,Sads,Angrys,Updatetime,Link
0,亞太電信Gt智慧生活,Aptg.tw,2019-04-22 20:53,【就是要全速！挑戰新低299 限時快閃價⚡️】 限時最後倒數，299強力全速飆！ 現在至亞太...,993,844,999,4,0,1,0,0,2019-05-28 15:49,https://www.facebook.com/Aptg.tw/photos/a.6983...
1,亞太電信Gt智慧生活,Aptg.tw,2019-04-26 20:30,網美出去玩第五站👉#新莊文昌嗣 準備考試的考生們辛苦了，不妨來到新莊文昌嗣求個學業平安吧 與...,7,0,38,2,0,0,0,0,2019-05-28 15:48,https://www.facebook.com/Aptg.tw/posts/2422597...
2,亞太電信Gt智慧生活,Aptg.tw,2019-04-30 18:01,There's No Text Content!,5,0,27,3,0,0,0,0,2019-05-28 15:47,https://www.facebook.com/Aptg.tw/photos/a.3758...
3,亞太電信Gt智慧生活,Aptg.tw,2019-04-28 02:17,網美出去玩第八站👉#2019台灣文博會 搖身一跳，小編來到文青一定要到的文博會～ 總共有三個...,5,0,52,1,0,1,0,0,2019-05-28 15:47,https://www.facebook.com/Aptg.tw/posts/2424854...
4,亞太電信Gt智慧生活,Aptg.tw,2019-05-01 20:00,📣 5月繳稅季 #亞太電信好康3連發 與您義氣相挺 報稅季開跑，國民們盡義務的時間又到了😭 ...,3800,1000,12000,99,29,0,0,0,2019-05-28 15:46,https://www.facebook.com/Aptg.tw/photos/a.1653...
5,亞太電信Gt智慧生活,Aptg.tw,2019-05-04 20:00,不用上班上課的日子，就是～處去玩的日子 網美出去玩第九站👉石門 老梅綠石槽 依附在石槽上的海...,5,0,42,1,1,0,0,0,2019-05-28 15:38,https://www.facebook.com/Aptg.tw/photos/a.1653...
6,亞太電信Gt智慧生活,Aptg.tw,2019-05-06 04:00,GtPay蛋蛋好馨情❤5月嗶嗶超有愛❤ 用GtPay購物繳費嗶一下，茶葉蛋請你吃❗ 奪蛋三步...,3,0,42,1,0,0,0,0,2019-05-28 15:37,https://www.facebook.com/Aptg.tw/photos/a.1653...
7,亞太電信Gt智慧生活,Aptg.tw,2019-05-08 04:00,媽～你的禮物在這裡🎁 迫在眉睫，禮物竟然還沒準備好❓❗ 別慌張，來亞太電信一次幫你搞定😎 愛...,2,2,40,1,0,0,0,0,2019-05-28 15:37,https://www.facebook.com/Aptg.tw/photos/a.1653...
8,亞太電信Gt智慧生活,Aptg.tw,2019-05-08 20:00,一步驟❗抽 iPhone XR 手機🎉 只要….來門市e起愛地球，節能使用e帳單 108/5...,7,4,46,3,0,0,0,0,2019-05-28 15:36,https://www.facebook.com/Aptg.tw/photos/a.1653...
9,亞太電信Gt智慧生活,Aptg.tw,2019-05-09 04:00,就決定是你了‼ 皮卡丘，幫我手機充電一下😅 手機即將沒電又沒得充，與其呼喊神隊友-皮卡丘 不...,16,6,48,2,4,0,0,0,2019-05-28 15:36,https://www.facebook.com/Aptg.tw/photos/a.1653...


In [11]:
Comments = Comments.reset_index(drop=True)
Comments = Comments.loc[:,['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName', 'Link', 'Updatetime']]
Comments

,ID,Name,Time,Content,RepID,RepName,Link,Updatetime
0,100026092540132,賴溫榮,2019-04-23 18:14,戴余秦 戴岳廷 Griffith Luo「亞太電信限時299 全速瘋起來 申辦就要快」。,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.6983...,2019-05-28 15:49
1,100001822494306,莫秋璃,2019-04-25 15:37,「陳隆隆 林依晨 金釉拉 賴欀欀 亞太電信限時299 全速瘋起來 申辦就要快」,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.6983...,2019-05-28 15:49
2,100003734253244,謝佳縈,2019-04-24 00:53,Yu Kimi 姿穎何 陳柏瑞 謝依美「亞太電信限時299 全速瘋起來 申辦就要快」,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.6983...,2019-05-28 15:49
3,100031026543314,蔡佑佑,2019-04-28 12:26,Popo Peng 蔡小潔 陳真臻「亞太電信限時299 全速瘋起來 申辦就要快」。,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.6983...,2019-05-28 15:49
4,100026352129711,王王鳳,2019-04-23 21:18,Mu Mu Chang 尹惠尹 張杏「亞太電信限時299 全速瘋起來 申辦就要快」,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.6983...,2019-05-28 15:49
5,100026047623143,姜中維,2019-04-28 21:44,Sung Po JJ 簡碧梅 姜彦彥「亞太電信限時299 全速瘋起來 申辦就要快」,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.6983...,2019-05-28 15:49
6,100002658981910,黃彩鳳,2019-04-23 14:05,D.h. Lin 張小甄 黃建源「亞太電信限時299 全速瘋起來 申辦就要快」,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.6983...,2019-05-28 15:49
7,100000363105519,李武鴻,2019-04-23 23:46,「 傅賢良 ，陳茹茹 ，Hanna Liu 亞太電信限時299 全速瘋起來 申辦就要快」,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.6983...,2019-05-28 15:49
8,100024837975101,謝吉,2019-04-24 16:01,Fanta Ti 謝小婷 單樂 「亞太電信限時299 全速瘋起來 申辦就要快」,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.6983...,2019-05-28 15:49
9,100009804871377,李秀珠,2019-04-24 09:38,蔡潔葛 張佑 Ya Jhang「亞太電信限時299 全速瘋起來 申辦就要快」-///-,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.6983...,2019-05-28 15:49


In [12]:
Posts.to_pickle('C:/Users/TL_Yu/Desktop/Posts.plk')
Comments.to_pickle('C:/Users/TL_Yu/Desktop/Comments.plk')

In [24]:
#Posts.to_csv('C:/Users/TL_Yu/Desktop/Posts.csv')
#Comments.to_csv('C:/Users/TL_Yu/Desktop/Comments.csv')

# 維護測試區

In [13]:
driver = webdriver.Chrome()
driver.get('https://www.facebook.com/')
time.sleep(1)
driver.find_element_by_partial_link_text('English').click()

In [114]:
# https://www.facebook.com/tstartel/posts/3284730548219636
# https://www.facebook.com/tstartel/videos/1100239263492734/?permPage=1
driver.get('https://www.facebook.com/chtmobile/photos/a.264830403546609/2722896354406656/')
PostExpand()

This can not expand


In [26]:
soup = BeautifulSoup(driver.page_source)
CommentsInfo(soup)

,ID,Name,Time,Content,RepID,RepName,Link,Updatetime
0,100008036701674,陳國進,2019-05-26 12:44,我今天要去屏東市😆😆😆,360044337354953,台灣之星,https://www.facebook.com/tstartel/photos/a.854...,2019-05-27 01:14
1,360044337354953,台灣之星,2019-05-26 13:10,要做好防曬唷!!!,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.854...,2019-05-27 01:14
2,100000293657248,賴清溪,2019-05-26 13:16,貼心😏,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.854...,2019-05-27 01:14
3,360044337354953,台灣之星,2019-05-26 13:28,🤗,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.854...,2019-05-27 01:14
4,100008036701674,陳國進,2019-05-26 17:26,賴清溪 真的很貼心呢～,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.854...,2019-05-27 01:14
5,100008036701674,陳國進,2019-05-26 17:26,台灣之星 好哦～謝謝小編,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.854...,2019-05-27 01:14
6,360044337354953,台灣之星,2019-05-26 17:35,如何~今天有沒有看見什麼有趣的事呢~~~國進~~~,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.854...,2019-05-27 01:14
7,100008036701674,陳國進,2019-05-26 19:33,台灣之星 沒有~~但以上速度超穩~~,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.854...,2019-05-27 01:14
8,360044337354953,台灣之星,2019-05-26 19:41,期待你分享小祕密唷(〃ω〃),100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.854...,2019-05-27 01:14
9,100008036701674,陳國進,2019-05-26 19:48,台灣之星 下載80.5 上傳45.0,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.854...,2019-05-27 01:14


In [24]:
soup = BeautifulSoup(driver.page_source)

Crawl Comments Failed!


,ID,Name,Time,Content,RepID,RepName,Link
0,100008036701674,陳國進,2019-05-26 12:44,我今天要去屏東市😆😆😆,360044337354953,台灣之星,https://www.facebook.com/tstartel/photos/a.854...


In [56]:
PostInfo(soup)

,Name,ID,Time,Content,Comments,Shares,Likes,Loves,Hahas,Wows,Sads,Angrys,Updatetime,Link
0,台灣之星,tstartel,2019-05-15 22:11,台灣之星2019全新品牌主張 你的電信就該是這個樣子⚡今日登場 全民一起辦活動⚡同步開跑‼️...,1700,88,732,12,0,0,0,20,2019-05-27 00:43,https://www.facebook.com/tstartel/posts/327440...


In [57]:
CommentsInfo(soup)

,ID,Name,Time,Content,RepID,RepName,Link,Updatetime
0,100004563705267,李維尼,2019-05-16 13:14,Line 好友不是說299只到今天上午09:59。,360044337354953,台灣之星,https://www.facebook.com/tstartel/posts/327440...,2019-05-27 00:44
1,360044337354953,台灣之星,2019-05-16 13:25,我們聽到大家的心聲~加碼到5/31唷!!!!,100004563705267,李維尼,https://www.facebook.com/tstartel/posts/327440...,2019-05-27 00:44
2,100003595572374,魏坤輝,2019-05-16 13:33,網頁進不去,100004563705267,李維尼,https://www.facebook.com/tstartel/posts/327440...,2019-05-27 00:44
3,360044337354953,台灣之星,2019-05-16 13:57,小編測試可以唷~~http://bit.ly/2HihMLD,100004563705267,李維尼,https://www.facebook.com/tstartel/posts/327440...,2019-05-27 00:44
4,100004563705267,李維尼,2019-05-16 14:03,台灣之星 請問我去年有辦188了，但188是21m吃到飽，現在299是不限速吃到飽，我可以去...,100004563705267,李維尼,https://www.facebook.com/tstartel/posts/327440...,2019-05-27 00:44
5,360044337354953,台灣之星,2019-05-16 14:23,老客戶免費勁速體驗底家~快去申請體驗吧~https://doc.tstartel.com/e...,100004563705267,李維尼,https://www.facebook.com/tstartel/posts/327440...,2019-05-27 00:44
6,100004563705267,李維尼,2019-05-16 14:26,台灣之星 謝謝，已申請，OK的話，馬上來辦。,100004563705267,李維尼,https://www.facebook.com/tstartel/posts/327440...,2019-05-27 00:44
7,100004563705267,李維尼,2019-05-16 14:39,台灣之星 看來我家是各大電信悲劇，競速是23.9/11.6，和188吃到飽，差不多。,100004563705267,李維尼,https://www.facebook.com/tstartel/posts/327440...,2019-05-27 00:44
8,360044337354953,台灣之星,2019-05-16 14:50,行動網路容易受使用地點附近的房屋密集度、建築裝潢使用材質、附近使用人數多寡…等等因素影響，我...,100004563705267,李維尼,https://www.facebook.com/tstartel/posts/327440...,2019-05-27 00:44
9,100001860123628,Wu Hung Chuan,2019-05-16 13:15,499沒有比較划算，你們的費率本來就超過1分1元，會打到499的算起來還是499為什麼要辦4...,360044337354953,台灣之星,https://www.facebook.com/tstartel/posts/327440...,2019-05-27 00:44


In [55]:
Posts = UpdateData(DateFrame_o = Posts, DateFrame_n = nPost)
Posts

,Name,ID,Time,Content,CommentCounts,Shares,Like,Love,Haha,Wow,Sad,Angry,Updatetime,Link
2,台灣之星,360044337354953,2019-05-18 21:00,好想去日本🇯🇵看繡球花唷 小編們聽 #U姐編 嘟嘟囔囔一週快煩死了 交換個眼色，作戰計劃開始...,55,13,626,8,2,0,0,0,2019-05-20 23:47,https://www.facebook.com/tstartel/photos/a.413...
0,台灣之星,360044337354953,2019-05-19 02:00,#週1福利日 #每週一1230 #週週登場 有沒有人跟 #00C編 一樣 手機沒電會狂症發作...,13,4,299,3,0,5,0,0,2019-05-20 23:46,https://www.facebook.com/tstartel/photos/a.413...
1,台灣之星,360044337354953,2019-05-19 20:52,反正只要一兩塊錢 買個袋子裝一裝，比較方便啦😆 環保餐具用完還要洗，好麻煩 直接拿竹筷和塑膠...,35,13,79,1,0,0,3,0,2019-05-20 23:45,https://www.facebook.com/tstartel/posts/328473...


In [56]:
Comments = UpdateData(DateFrame_o = Comments,DateFrame_n = nComments)
Comments

,ID,Name,Time,Content,RepID,RepName,Link,Updatetime
48,360044337354953,台灣之星,2019-05-19 15:52,看美美照片一樣心情會hen好,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
61,360044337354953,台灣之星,2019-05-19 19:40,img,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
70,100008036701674,陳國進,2019-05-20 17:18,台灣之星 還好我16:00回到家過一下才下雨~~,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
69,360044337354953,台灣之星,2019-05-20 16:48,有沒有記得帶雨傘呀~,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
68,100008036701674,陳國進,2019-05-20 16:44,台灣之星 小編，下大雨了,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
67,360044337354953,台灣之星,2019-05-19 21:08,加油加油(๑˃̵ᴗ˂̵)ﻭ其實上課是很幸福D~~晚安😉😉,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
66,100008036701674,陳國進,2019-05-19 21:03,台灣之星 明天又要上課，小編晚安~~,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
65,360044337354953,台灣之星,2019-05-19 20:47,早點洗洗睡明天又是新的一天~(✿◠‿◠),100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
64,100008036701674,陳國進,2019-05-19 20:42,台灣之星 是啊～,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
63,360044337354953,台灣之星,2019-05-19 20:41,國進到家啦❓,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47


In [16]:
driver.quit()

# 障礙清單
https://www.facebook.com/taiwanmobile/photos/a.1515817481972021/2484327321787694/

# 其他

In [18]:
# 原本想要登入的方式獲取連結，但FB會偵測異常行為，暫時先不登入
# 關閉Chrome的「通知」提醒
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-notifications")

driver = webdriver.Chrome(options=chrome_options)
url = 'https://www.facebook.com/'
driver.get(url)
time.sleep(3)

# 啟動瀏覽器並登入Facebook
username = driver.find_element_by_id('email')
username.send_keys('61034b001@gms.ndhu.edu.tw')
passwd=driver.find_element_by_id('pass')
passwd.send_keys('19920309')
button=driver.find_element_by_id('loginbutton')
button.click()